# NIfTI to DICOM-RTSS Conversion

In this example, we demonstrate the conversion from NIfTI to DICOM-RTSS using the example data provided (details see [Examples](examples.rst). Because DICOM-RTSS files are closely linked to the corresponding DICOM image series and include references to those, the conversion procedure is not as straightforward as it may be assumed. However, PyRaDiSe provides all necessary functionality and automates many of the tasks, such that the user requires minimal effort.

Before going through this example, we want to emphasize that this conversion procedure will fail if the input NIfTI data is not within the extent of the corresponding DICOM image series. The conversion routines are not feasible to compensate for data misalignment because this is up to the user.


## Import Procedure
First of all, let's import the necessary classes for this example.

In [1]:
import os
from typing import Optional

from pyradise.data import (Modality, Organ, Rater)
from pyradise.fileio import (SubjectLoader, DatasetFileCrawler, SubjectDicomCrawler,
                             ModalityExtractor, OrganExtractor, RaterExtractor, Tag,
                             NoRTSSInfoSelector, SubjectToRTSSConverter,
                             DicomSeriesSubjectWriter)

## Extractor Implementation
Because the naming of NIfTI files is flexible and does not incorporate content-related metadata, PyRaDiSe requires the user to implement `Extractors` to retrieve information about the `Modality` of the `IntensityImages` or, in the case of `SegmentationImages` about the `Organ` depicted and the `Rater` who created the segmentations. The approach of using user-implemented `Extractors` allows for maximum flexibility because the necessary content-related information can be either retrieved from the file path of the corresponding file or by looking into a third-party information source such as a database.

For implementation, one must inherit from the provided `Extractor` base classes that provide implemented examples and explanations. In general, each `Extractor` gets the file path of the NIfTI file for which the content-related information is requested. Based on this path, the user must implement the `extract` method such that the method returns the requested information or `None` if the image type does not fit the `Extractor`.

The `ModalityExtractor`, in contrast to other `Extractor` types, possesses two `extract` methods, which are separately used for DICOM data and discrete medical images. If the user exclusively works which DICOM or discrete image data, the other `extract` method may skip from implementation by returning None. However, both `extract` methods are required in this example because DICOM and discrete image data are processed.

In [2]:
class ExampleModalityExtractor(ModalityExtractor):

    def extract_from_dicom(self, path: str) -> Optional[Modality]:
        # Extract the necessary attributes from the DICOM file
        tags = (Tag(0x0008, 0x0060),  # Modality
                Tag((0x0008, 0x103e)))  # Series Description
        dataset_dict = self._load_dicom_attributes(tags, path)

        # Identify the modality rule-based
        modality = dataset_dict.get('Modality', {}).get('value', None)
        series_desc = dataset_dict.get('Series Description', {}).get('value', '')
        if modality == 'MR':
            if 't1' in series_desc.lower():
                return Modality('T1')
            elif 't2' in series_desc.lower():
                return Modality('T2')
            else:
                return None
        else:
            return None

    def extract_from_path(self, path: str) -> Optional[Modality]:
        # Identify the discrete image file's modality rule-based
        filename = os.path.basename(path)

        # Check if the image contains an img prefix
        # (i.e., it is a intensity image)
        if not filename.startswith('img'):
            return None

        # Check if the image contains a modality search string
        if 'T1' in filename:
            return Modality('T1')
        elif 'T2' in filename:
            return Modality('T2')
        else:
            return None


class ExampleOrganExtractor(OrganExtractor):

    def extract(self, path: str) -> Optional[Organ]:
        # Identify the discrete image file's organ rule-based
        filename = os.path.basename(path)

        # Check if the image contains a seg prefix
        # (i.e., it is a segmentation)
        if not filename.startswith('seg'):
            return None

        # Split the filename for extracting the organ name
        organ_name = filename.split('_')[-1].split('.')[0]
        return Organ(organ_name)


class ExampleRaterExtractor(RaterExtractor):

    def extract(self, path: str) -> Optional[Rater]:
        # Identify the discrete image file's rater rule-based
        filename = os.path.basename(path)

        # Check if the image contains a seg prefix
        # (i.e., it is a segmentation)
        if not filename.startswith('seg'):
            return None

        # Split the filename for extracting the rater name
        rater_name = filename.split('_')[2]
        return Rater(rater_name)

## Conversion Procedure Construction
In the following code section, one defines first a reference `Modality` used for identifying the reference DICOM image series. Then, several specific component instantiations follow this, establishing the functionality for crawling, loading, selecting `SeriesInfo` entries, and writing the newly generated DICOM-RTSS. After setting up these functional components, the NIfTI `Crawler` extracts subject-by-subject the necessary `SeriesInfo` entries using the antecedently implemented `Extractors`. Following the NIfTI crawling, the NIfTI data gets loaded using the `SubjectLoader`. After executing the crawling of the DICOM data and a selection process on the `SeriesInfo` for excluding the old DICOM-RTSS data, conversion of the NIfTI data takes place. Finally, the writing of the resulting DICOM-RTSS dataset is achieved using the `DicomSeriesSubjectWriter`'s functionality.

In [3]:
def convert_subject_to_dicom_rtss(input_dir_path: str,
                                  output_dir_path: str,
                                  dicom_image_dir_path: str
                                  ) -> None:
    # Specify a reference modalities
    # This is the modality of the DICOM image series that will be
    # referenced in the DICOM-RTSS.
    reference_modality = Modality('T1')

    # Create the loader
    loader = SubjectLoader()

    # Create the writer and specify the output file name of the
    # DICOM-RTSS files
    writer = DicomSeriesSubjectWriter()
    rtss_filename = 'rtss.dcm'

    # (optional)
    # Instantiate a new selection to exclude the original DICOM-RTSS SeriesInfo
    # Note: If this is omitted the original DICOM-RTSS will be copied to the
    # corresponding output directory.
    selection = NoRTSSInfoSelector()

    # Create the file crawler for the discrete image files and
    # loop through the subjects
    crawler = DatasetFileCrawler(input_dir_path,
                                 extension='.nii.gz',
                                 modality_extractor=ExampleModalityExtractor(),
                                 organ_extractor=ExampleOrganExtractor(),
                                 rater_extractor=ExampleRaterExtractor())
    for series_info in crawler:

        # Load the subject
        subject = loader.load(series_info)

        # Construct the path to the subject's DICOM images
        dicom_subject_path = os.path.join(dicom_image_dir_path, subject.get_name())

        # Construct a DICOM crawler to retrieve the reference
        # DICOM image series info
        dcm_crawler = SubjectDicomCrawler(dicom_subject_path,
                                          modality_extractor=ExampleModalityExtractor())
        dicom_series_info = dcm_crawler.execute()

        # (optional)
        # Keep all SeriesInfo entries that do not describe a DICOM-RTSS for loading
        dicom_series_info = selection.execute(dicom_series_info)

        # Convert the segmentations to a DICOM-RTSS with standard smoothing settings
        # Note: Too strong smoothing may lead to corrupted structures if their size
        # is too small
        converter = SubjectToRTSSConverter(subject, dicom_series_info,
                                           reference_modality)
        rtss = converter.convert()

        # Combine the DICOM-RTSS with its output file name
        rtss_combination = ((rtss_filename, rtss),)

        # Write the DICOM-RTSS to a separate subject directory
        # and include the DICOM files crawled before
        # Note: If you want to output just a subset of the
        # original DICOM files you may use additional selectors
        writer.write(rtss_combination, output_dir_path,
                     subject.get_name(), dicom_series_info)


## Execute Conversion Procedure

Before executing the implemented procedure, one needs to adjust the following paths accordingly. Make sure that the output directory is empty because PyRaDiSe is not allowed to override existing files to reduce the risks of data loss. Afterward, the conversion procedure can be executed, and the results should appear in the specified output directory.

In [4]:
# The input path pointing to the top-level directory containing the
# NIfTI subject directories
input_dataset_path = 'C:/temp/test_output_6'

# The input path pointing to the top-level directory containing the
# DICOM subject directories that will get referenced in the output
# DICOM-RTSS files
dicom_dataset_path = 'D:/tcia_data/curated_data/'

# The output path pointing to an empty directory where the output
# will be saved
output_dataset_path = 'C:/temp/rtss_output_6'

# Execution of the conversion procedure
convert_subject_to_dicom_rtss(input_dataset_path,
                              output_dataset_path,
                              dicom_dataset_path)

## Expected Result
After the conversion procedure has been executed, the following output should result using the provided example data. The following image, taken from subject VS-SEG-001, shows overlayed tumor volume segmentations from the input NIfTI file (green) and the newly generated DICOM-RTSS (red).

<p style="text-align:center;"><img src="../examples/conversion/images/nii_to_dicom_res_0.png"  height="500"></p>

The following skull reconstruction of subject VS-SEG-001 from the example data, demonstrates the overlap between the original DICOM data (green) and the converted DICOM-RTSS (red) that was previously converted to NIfTI.

<p style="text-align:center;"><img src="../examples/conversion/images/nii_to_dicom_res_1.png"  height="500"></p>

Please note the missing segmentation volume at the nose tip of the subject. This issue demonstrates what happens if the NIfTI segmentation is not entirely inside the DICOM image and thus can not be reconstructed.